# Widgets Overview

The following is a run through of the widgets you can create with Interact. All widgets allow for the following keyword arguments:

* `label`: Label to be shown next to the widget
* `value`: The value the widget should be set to when created
* `signal`: A signal object of type `Reactive.Input` which gets the value of the widget as user enters input.

Many of the widgets have keyword arguments specific to them. See below for more.

In [1]:
using Interact
using Reactive

## Slider

Sliders are arguably the most useful of the widgets. A slider can be created with the `slider{T <: Number}(range::Range{T})` function. The value of the slider defaults to the median of the range, and can be set using the `value::T` keyword argument. The type of signal a slider depends on the type of the range. E.g. A floating point range like `0:π/4:2π` gives a signal of floating point values, while a range like `1:10` gives a signal of integers.

In [2]:
float_slider = slider(0:π/4:2π)

Interact.Slider{Float64}(Signal{Float64}(3.141592653589793, nactions=0),"",3.141592653589793,0.0:0.7853981633974483:6.283185307179586,true)

In [3]:
int_slider = slider(1:10)

Interact.Slider{Int64}(Signal{Int64}(5, nactions=0),"",5,1:10,true)

In [5]:
signal(int_slider)

6

## Checkbox

`checkbox` takes an optional first argument which defaults to `false` and creates a checkbox.

In [6]:
display(checkbox())
checkbox(true)

Interact.Checkbox(Signal{Bool}(false, nactions=0),"",false)

Interact.Checkbox(Signal{Bool}(true, nactions=0),"",true)

## Toggle

You can create a toggle button with `togglebutton` which takes as an optional argument its label.

In [7]:
status = togglebutton("Mary called", value=true)

Interact.ToggleButton(Signal{Bool}(true, nactions=0),"Mary called",true)

In [9]:
map(s -> s ? "Mary called" : "Mary didn't call", signal(status))

"Mary called"

## Button

A button gives out a signal of a constant signal which is `nothing` by default. You can set this using the `value` keyword argument. The signal updates when the button is clicked. 

In [14]:
b = button("Click Me")

Interact.Button{Void}(Signal{Void}(nothing, nactions=0),"Click Me",nothing)

Here is how you can count the number of clicks made on a button using `foldp` on the signal, (think of `foldp` as fold over past values):

In [15]:
foldp((acc, value) -> acc + 1, 0, signal(b))

0

## Option widgets

There are 3 options widgets: `dropdown`, `togglebuttons`, `radiobuttons`. There are two types allowed as an argument while invoking these:
1. `AbstractArray` (e.g. `Vector`, `Tuple`)
2. `Associative` (e.g. `Dict`, `OrderedDict`)
The default value is the first element (or undefined in case of an undordered `Associative` like `Dict`), but this can be set using the `value` keyword argument.

In [16]:
a = dropdown(["one", "two", "three"])

Interact.Options{:Dropdown,ASCIIString}(Signal{ASCIIString}(one, nactions=0),"","one","one",Interact.OptionDict(Any["one","two","three"],Dict{Any,Any}("two"=>"two","one"=>"one","three"=>"three")),Any[],Any[])

In [17]:
signal(a)

"two"

In [20]:
f = radiobuttons(Dict("Add" => +, "Sub" => -, "Exp" => ^))

Interact.Options{:RadioButtons,Function}(Signal{Function}(-, nactions=0),"",-,"Sub",Interact.OptionDict(Any["Sub","Add","Exp"],Dict{Any,Any}("Sub"=>-,"Add"=>+,"Exp"=>^)),Any[],Any[])

In [22]:
map(g -> g(e, π*im), signal(f))

2.718281828459045 - 3.141592653589793im

Notice that the order "Add", "Sub", "Exp" was not retained in the above example, because a `Dict` does not save the ordering. To overcome this, we can use `OrderedDict` from DataStructures.jl package.

In [25]:
f_ = togglebuttons([("Add", +), ("Sub", -), ("Exp", ^)])

Interact.Options{:ToggleButtons,Function}(Signal{Function}(+, nactions=0),"",+,"Add",Interact.OptionDict(Any["Add","Sub","Exp"],Dict{Any,Any}("Sub"=>-,"Add"=>+,"Exp"=>^)),Any[],Any[])

In [27]:
map(g -> g(e, π*im), signal(f_))

-1.0 + 1.2246467991473532e-16im

## Textbox

A textbox can be of a `Number` or `AbstractString` type. `textbox` takes one argument: its default value.

In [28]:
string_box = textbox("Change me")

Interact.Textbox{UTF8String}(Signal{UTF8String}(Change me, nactions=0),"",nothing,"Change me")

In [29]:
signal(string_box)

"Change me"

A textbox can be of a `Number` type as well. Just set a default number value, or use `textbox(typ=T)` where T is a `Number` type.

In [30]:
int_box = textbox(0)

Interact.Textbox{Int64}(Signal{Int64}(0, nactions=0),"",nothing,0)

In [31]:
signal(int_box)

0

If creating a number typed textbox, you can also pass along an optional `range` field to set a bound on the possible values one can input. If an entered value exceeds the range, it is replaced by its nearest bounding number.

In [32]:
bounded_float_box = textbox(2pi, range=-10.0:10)

Interact.Textbox{Float64}(Signal{Float64}(6.283185307179586, nactions=0),"",-10.0:1.0:10.0,6.283185307179586)

In [33]:
signal(bounded_float_box)

6.283185307179586

## Textarea

`textarea` takes an optional default value and creates a textarea. Its signal changes when you type.

In [34]:
tex = textarea("Your very own \$\\LaTeX\$ editor")

Interact.Textarea{ASCIIString}(Signal{ASCIIString}(Your very own $\LaTeX$ editor, nactions=0),"","Your very own \$\\LaTeX\$ editor")

In [36]:
map(latex, signal(tex))

Interact.Latex("","Your very own \$\\LaTeX\$ editor")

## The `widget` Function

`widget` tries to coerce a value into a widget.

In [38]:
map(display, [
    widget(1:10),                 # Slider
    widget(false),                # Checkbox
    widget("text"),               # Textbox
    widget(1.1),                  # Number Textbox
    widget([:on, :off]),          # Toggle Buttons
    widget(Dict(:π => float(π), :τ => 2π))
    ]);

Interact.Slider{Int64}(Signal{Int64}(5, nactions=0),"",5,1:10,true)

Interact.Checkbox(Signal{Bool}(false, nactions=0),"",false)

Interact.Textbox{AbstractString}(Signal{AbstractString}(text, nactions=0),"",nothing,"text")

Interact.Textbox{Float64}(Signal{Float64}(1.1, nactions=0),"",nothing,1.1)

Interact.Options{:ToggleButtons,Symbol}(Signal{Symbol}(on, nactions=0),"",:on,"on",Interact.OptionDict(Any["on","off"],Dict{Any,Any}("off"=>:off,"on"=>:on)),Any[],Any[])

Interact.Options{:ToggleButtons,Float64}(Signal{Float64}(6.283185307179586, nactions=0),"",6.283185307179586,"τ",Interact.OptionDict(Any["τ","π"],Dict{Any,Any}("τ"=>6.283185307179586,"π"=>3.141592653589793)),Any[],Any[])